<a href="https://colab.research.google.com/github/IvlevaAnna1/Hometasks/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%BA_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8E_4_Ivleva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практика
Используемые библиотеки

In [ ]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [179]:
# headers={
#     'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
# }
# def get_content(url):
#     with requests.Session() as req:
#         req.headers.update(headers)
#         r = req.get(url).content
#     return r

# url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
# s = get_content(url)
# df=pd.read_csv(io.StringIO(s.decode('UTF8')))


### Если не работает загрузка on-line
df=pd.read_csv('/content/drive/MyDrive/Him_analiz.csv', sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [180]:
# Ваше решение
df.rename(columns={'Показатель': 'Indicator','Единица измерений': 'Unit','Результат анализа': 'Result','Норматив': 'Norma'}, inplace=True)
df['Result'] = df['Result'].replace('б/цвета', '0')
df['Norma'] = df['Norma'].str.replace(',', '.')
df

,Indicator,Unit,Result,Norma
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,0,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,не более 1.5
5,Нитриты (по NO2),мг/дм3,0.017,не более 3.3
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,не более 3.5
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [181]:
list = []
for index, row in df.iterrows():
  if "в пределах" in row['Norma']:
    if float(str(row['Norma'].replace('в пределах ','').split('-')[0])) <= float(str(row['Result'])) <= float(str(row['Norma'].replace('в пределах ','').split('-')[1])):
      list.append("Показатель в норме")
    elif float(str(row['Norma'].replace('в пределах ','').split('-')[0])) > float(str(row['Result'])):
      list.append("Показатель ниже нормы")
    elif float(str(row['Norma'].replace('в пределах ','').split('-')[1])) > float(str(row['Result'])):
      list.append("Показатель выше нормы")
  elif "не более " in row['Norma']:
    if float(str(row['Norma'].replace('не более ','').split('-')[-1])) >= float(str(row['Result'])):
      list.append("Показатель в норме")
    elif float(str(row['Norma'].replace('не более ',''))) < float(str(row['Result'])):
      list.append("Показатель выше нормы")

In [182]:
list

['Показатель в норме',
 'Показатель в норме',
 'Показатель в норме',
 'Показатель в норме',
 'Показатель в норме',
 'Показатель в норме',
 'Показатель в норме',
 'Показатель в норме',
 'Показатель в норме',
 'Показатель в норме',
 'Показатель в норме',
 'Показатель выше нормы',
 'Показатель в норме',
 'Показатель выше нормы']

In [183]:
df.insert(loc= len(df.columns) , column='Description', value=list)

In [184]:
df

,Indicator,Unit,Result,Norma,Description
0,pH,единицы pH,8.4,в пределах 6-9,Показатель в норме
1,Запах,баллы,1,не более 2-3,Показатель в норме
2,Цветность,градусы,0,не более 30,Показатель в норме
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Показатель в норме
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,не более 1.5,Показатель в норме
5,Нитриты (по NO2),мг/дм3,0.017,не более 3.3,Показатель в норме
6,Нитраты (по NO3),мг/дм3,24,не более 45,Показатель в норме
7,Фосфаты (P),мг/дм3,0.36,не более 3.5,Показатель в норме
8,Хлориды (Cl),мг/дм3,200,не более 350,Показатель в норме
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,Показатель в норме


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [ ]:
# Ваше решение

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
# Ваше решение